In [4]:
import mikeio
import numpy as np
import pandas as pd
import datetime as dt
from res1d_v5 import *
import pickle
import plotly.express as px
from IPython.display import clear_output

Brainstorm

Add flood zoomable floodmap
Add 1s result timestep for pump stations


In [10]:
#General variables
periods = []
periods.append('DWF')
for i in range(1,6):
    periods.append('WWF' + str(i))
periods

['DWF', 'WWF1', 'WWF2', 'WWF3', 'WWF4', 'WWF5']

In [5]:
#Model specific variables
model_area = "NSSA"

calibration_sheet = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Calibration_Master.xlsm"
rainfall_dfs0_file = ""

map_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Report_Maps"
output_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Jupyter_Reports"
result_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\06. CALIB_RESULTS\Latest_Backup49"

dfs0_folders = []
dfs0_folders.append(r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\06_FLOW MONITORING\10. Dfs0s\Non-PS")
dfs0_folders.append(r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\06_FLOW MONITORING\10. Dfs0s\PS")

In [22]:
# sample_list is defined list
sample_list = [1,2,3,'seeker',3,7.5,3,3,3,3,9]
# set() to convert list to set
sample_set  = set(sample_list)
print(sample_set) #printing set

{1, 2, 3, 7.5, 'seeker', 9}


In [43]:
gauges = pd.read_excel(calibration_sheet,sheet_name="Gauges")
gauges = gauges.loc[:, :'Level Type']
gauges = gauges[gauges.Model.str.contains(model_area)]
gauges

,Gauge,Pipe,CSO1,CSO2,Location,Node1 (Or Pipe if pipe level),Node2,CSOPipe1,CSOPipe2,Rain Gauge,Model,Sensor Output,Location Type,Pipe Shape,Pipe Dimension,Node1 AssetName,Flow source,Level Type
223,FA2,49582,NaN,NaN,FA2,9600,NaN,NaN,NaN,DN72,NSSA-E,Level and Flow,Gauge,Circular,0.6,MH12A,Measured,Node
224,GE1,50924,NaN,NaN,GE1,GE1-WetWell,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,PS,Circular,0.2,,Measured,Node
225,GE2,56296,NaN,NaN,GE2,GE2-WetWell,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,PS,Circular,0.2,NaN,Measured,Node
226,GE3,50933,NaN,NaN,GE3,GE3-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
227,GE4,50940,NaN,NaN,GE5,GE4-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
228,GE5,50952,NaN,NaN,GE5,GE5-WetWell,NaN,NaN,NaN,VW49,NSSA-W,Level and Flow,PS,Circular,0.25,,Measured,Node
229,GN14,49709,NaN,NaN,GN14,7169,NaN,NaN,NaN,VW67,NSSA-W,Level and Flow,Gauge,Circular,0.45,MH14,Measured,Node
230,HL1,43953,NaN,NaN,HL1,9721,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.9,MH3,Measured,Node
231,HL14,43945,NaN,NaN,HL14,9712,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.75,MH14,Measured,Node
232,HL18,49559,NaN,NaN,HL14,9708,NaN,NaN,NaN,VW14,NSSA-W,Level and Flow,Gauge,Circular,0.55,MH18,Measured,Node


In [44]:
periods = pd.read_excel(calibration_sheet,sheet_name="Periods")
periods = periods[periods.Model.str.contains(model_area)]

In [50]:
nodes1 = gauges[gauges['Level Type']=='Node']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
nodes2 = gauges[gauges['Level Type']=='Node']['Node2'].dropna().unique().flatten().tolist()
nodes = nodes1 + nodes2
#Convert all to string
nodes = [str(x) for x in nodes]
#Make set to clear duplicates
nodes = set(nodes)

pipes1 = gauges[gauges['Level Type'].str.contains('Link')]['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
pipes2 = gauges.Pipe.dropna().unique().flatten().tolist()
pipes = pipes1 + pipes2
#Convert all to string
pipes = [str(x) for x in pipes]
#Make set to clear duplicates
pipes = set(pipes)

In [51]:
pipes

{'40472',
 '41250',
 '41295',
 '41304',
 '43611',
 '43648',
 '43945',
 '43946',
 '43953',
 '43987',
 '43989',
 '43999',
 '47391',
 '47438',
 '49559',
 '49580',
 '49582',
 '49592',
 '49640',
 '49652',
 '49709',
 '49727',
 '50870',
 '50924',
 '50932',
 '50933',
 '50940',
 '50952',
 '54287',
 '54850',
 '54852',
 '56296',
 '56393',
 '56394_3',
 '56401',
 '56405',
 'Link_15',
 'Lynn-Total',
 'MKO_Overflow'}